In [ ]:
import os
import cv2
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import glob
import zipfile

In [ ]:
"""
#--------------------------------------------------
# 【提出に関して】kaggleのAPIキーを入れるのは無理なので、
# 一時的に実行可能な形にリファクタリングします。
# -------------------------------------------------
# Kaggleデータセットのダウンロード（初回のみ）
def download_kaggle_dataset():
       print("Kaggleデータセットをダウンロード中...")

    # Kaggle APIの設定
    !pip install kaggle
    !mkdir -p ~/.kaggle
    !cp /content/drive/MyDrive/kaggle.json ~/.kaggle/
    !chmod 600 ~/.kaggle/kaggle.json

    # データセットをダウンロード
    !kaggle datasets download -d koryakinp/fingers

    # ZIPファイルを解凍
    !unzip -q fingers.zip -d /content/kaggle_fingers/

    print("ダウンロード完了")
    """

In [ ]:
def load_image_resized(image_path, target_size=(64, 64)):
    """画像を読み込んでリサイズ"""
    image = cv2.imread(image_path)
    if image is None:
        return None

    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, target_size)
    return resized.flatten()

In [ ]:
def load_your_dataset(base_folder):
    """データセットを読み込み"""
    print("=== データセットを読み込み中 ===")
    X = []
    y = []

    if not os.path.exists(base_folder):
        print(f"警告: {base_folder} が見つかりません")
        return np.array([]), np.array([])

    subfolders = [d for d in os.listdir(base_folder)
                  if os.path.isdir(os.path.join(base_folder, d))]

    for folder_name in subfolders:
        finger_count = int(folder_name.split('-')[0])
        folder_path = os.path.join(base_folder, folder_name)

        image_files = glob.glob(os.path.join(folder_path, "*.jpg")) + \
                     glob.glob(os.path.join(folder_path, "*.png"))

        print(f"フォルダ {folder_name} (指の本数: {finger_count}) - 画像数: {len(image_files)}")

        for image_path in image_files:
            image_data = load_image_resized(image_path)
            if image_data is not None:
                X.append(image_data)
                y.append(finger_count)

    print(f"あなたのデータセット: {len(X)}枚読み込み完了")
    return np.array(X), np.array(y)


In [ ]:
import re

def load_kaggle_dataset(kaggle_folder="/content/kaggle_fingers"):
    """Kaggleデータセットを読み込み（ファイル名からラベル抽出）"""
    print("=== Kaggleデータセットを読み込み中 ===")
    X = []
    y = []

    target_folders = ["train", "fingers"]  # 利用対象（

    for folder_name in target_folders:
        folder_path = os.path.join(kaggle_folder, folder_name)
        if not os.path.exists(folder_path):
            print(f"警告: {folder_path} が存在しません")
            continue

        image_files = glob.glob(os.path.join(folder_path, "*.png")) + \
                      glob.glob(os.path.join(folder_path, "*.jpg")) + \
                      glob.glob(os.path.join(folder_path, "*.jpeg"))

        print(f"{folder_name}/: 画像数: {len(image_files)}")

        for image_path in image_files:
            filename = os.path.basename(image_path)
            match = re.search(r"_([0-5])[RL]\.", filename)
            if match:
                finger_count = int(match.group(1))
                image_data = load_image_resized(image_path)
                if image_data is not None:
                    X.append(image_data)
                    y.append(finger_count)
            else:
                print(f"ラベル抽出失敗: {filename}")

    print(f"Kaggleデータセット: {len(X)}枚読み込み完了")
    return np.array(X), np.array(y)


In [ ]:
def combine_datasets(X1, y1, X2, y2):
    """2つのデータセットを結合"""
    if len(X1) == 0:
        return X2, y2
    if len(X2) == 0:
        return X1, y1

    X_combined = np.vstack([X1, X2])
    y_combined = np.hstack([y1, y2])

    print(f"結合後のデータセット: {len(X_combined)}枚")
    print(f"クラス別データ数:")
    for finger_count in sorted(set(y_combined)):
        count = np.sum(y_combined == finger_count)
        print(f"  {finger_count}本指: {count}枚")

    return X_combined, y_combined

In [ ]:
def train_combined_model():
    """結合データセットでモデルを訓練"""

    # 1. あなたのデータセットを読み込み
    your_data_folder = "/content/frames"
    X_yours, y_yours = load_your_dataset(your_data_folder)

    # 2. Kaggleデータセットを読み込み
    X_kaggle, y_kaggle = load_kaggle_dataset()

    # 3. データセットを結合
    X_combined, y_combined = combine_datasets(X_yours, y_yours, X_kaggle, y_kaggle)

    if len(X_combined) == 0:
        print("エラー: データが読み込めませんでした")
        return None

    print(f"\n=== モデル訓練開始 ===")
    print(f"総データ数: {len(X_combined)}枚")

    # データ分割
    X_train, X_test, y_train, y_test = train_test_split(
        X_combined, y_combined, test_size=0.2, random_state=42, stratify=y_combined
    )

    print(f"訓練用: {len(X_train)}枚, テスト用: {len(X_test)}枚")

    # モデル訓練（より大きなデータセットに対応）
    model = RandomForestClassifier(
        n_estimators=200,  # 木の数を増加
        max_depth=15,      # 深さを増加
        min_samples_split=5,
        min_samples_leaf=2,
        random_state=42,
        n_jobs=-1
    )

    print("モデル訓練中...")
    model.fit(X_train, y_train)

    # 評価
    print("\n=== モデル評価 ===")
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, y_pred)

    print(f"テストセット精度: {accuracy:.3f}")
    print("\n詳細な分類レポート:")
    print(classification_report(y_test, y_pred))

    # あなたのデータだけでの性能も確認
    if len(X_yours) > 0:
        print("\n=== あなたのデータのみでの性能 ===")
        your_indices = np.arange(len(X_yours))
        if len(your_indices) > 10:  # 十分なデータがある場合
            X_your_train, X_your_test, y_your_train, y_your_test = train_test_split(
                X_yours, y_yours, test_size=0.3, random_state=42
            )
            y_your_pred = model.predict(X_your_test)
            your_accuracy = accuracy_score(y_your_test, y_your_pred)
            print(f"あなたのデータでの精度: {your_accuracy:.3f}")

    return model

In [ ]:
def save_combined_model(model):
    """結合学習したモデルを保存"""
    if model is None:
        return

    import pickle

    model_path = "/content/finger_model_combined_2.pkl"
    with open(model_path, 'wb') as f:
        pickle.dump(model, f)

    print(f"\n結合学習モデルを保存しました: {model_path}")

In [ ]:
# download_kaggle_dataset()

Kaggleデータセットをダウンロード中...
Dataset URL: https://www.kaggle.com/datasets/koryakinp/fingers
License(s): CC0-1.0
fingers.zip: Skipping, found more recently modified local copy (use --force to force download)
replace /content/kaggle_fingers/fingers/test/000e7aa6-100b-4c6b-9ff0-e7a8e53e4465_5L.png? [y]es, [n]o, [A]ll, [N]one, [r]ename: N
ダウンロード完了


In [ ]:
# メイン実行
if __name__ == "__main__":

    # モデル訓練
    combined_model = train_combined_model()

    # モデル保存
    save_combined_model(combined_model)

    print("\n=== 完了 ===")